In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [2]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [3]:
!pip install lm_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.8/238.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1

In [4]:
# Install required packages
!pip install accelerate bitsandbytes -i https://pypi.org/simple/

import torch

# Check for GPU
if not torch.cuda.is_available():
    raise RuntimeError("No GPU found. A GPU is needed for quantization.")

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from datasets import load_dataset, Dataset, DatasetDict
from unsloth import FastLanguageModel
from trl import SFTTrainer
from collections import Counter
import re
import json
import os
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
import time
import yaml





Looking in indexes: https://pypi.org/simple/
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [5]:
from lm_eval import evaluator
from lm_eval.tasks import TaskManager

In [6]:
import wandb

In [7]:
import lm_eval

os.environ["TOKENIZERS_PARALLELISM"] = "true"

TASKS_WE_USE = [
    {
        "name": "bbh_cot_fewshot_logical_deduction_three_objects",
        "num_shots": None,
        "is_gen": True,
        "in_openllm": False,
        "metric": "exact_match,get-answer",
    },
    {
        "name": "bbh_cot_fewshot_reasoning_about_colored_objects",
        "num_shots": None,
        "is_gen": True,
        "in_openllm": False,
        "metric": "exact_match,get-answer",
    },
    {
        "name": "bbh_cot_fewshot_temporal_sequences",
        "num_shots": None,
        "is_gen": True,
        "in_openllm": False,
        "metric": "exact_match,get-answer",
    },
]

TASK_TO_METRIC = {v["name"]: v["metric"] for v in TASKS_WE_USE}
TASK_TO_NUM_SHOT = {v["name"]: v["num_shots"] for v in TASKS_WE_USE}
ALL_TASKS = [v["name"] for v in TASKS_WE_USE]
GEN_TASKS = set([v["name"] for v in TASKS_WE_USE if v["is_gen"]])
OPENLLM_TASKS = set([v["name"] for v in TASKS_WE_USE if v["in_openllm"]])

@dataclass
class LMEvalArguments:
    model: str = field(
        default="hf",
        metadata={"help": "The model TYPE"},
    )
    model_name_or_path: str = field(
        default="unsloth/llama-2-7b-chat-bnb-4bit",
        metadata={"help": "The model name or path."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The model revision."},
    )
    tokenizer_name_or_path: str = field(
        default="",
        metadata={
            "help": "In some rare occasion, you may want to manually specify the tokenizer name or path. If empty, set to model_name_or_path."
        },
    )
    tokenizer_revision: str = field(
        default="main",
        metadata={"help": "The tokenizer revision."},
    )
    attn_implementation: str = field(
        default=None,
        metadata={"help": "The attention implementation."},
    )
    torch_dtype: str = field(
        default="auto",
        metadata={"help": "The torch dtype."},
    )
    trust_remote_code: bool = field(
        default=True,
        metadata={"help": "Whether to trust remote code."},
    )
    batch_size: int = field(
        default=8,
        metadata={"help": "The batch size."},
    )
    output_path: str = field(
        default="output",
        metadata={"help": "The output path for the results."},
    )
    log_samples: bool = field(
        default=True,
        metadata={"help": "Whether to log samples."},
    )
    verbosity: str = field(
        default="DEBUG",
        metadata={"help": "The verbosity level."},
    )
    to_wandb: bool = field(
        default=False,
        metadata={"help": "Whether to log to wandb."},
    )
    wandb_project: str = field(
        default="",
        metadata={"help": "The wandb project."},
    )
    wandb_id: str = field(
        default="",
        metadata={
            "help": "The wandb run id to upload results to. If empty, we will check the model_name_or_path/run_args.yaml"
        },
    )

In [8]:
def __post_init__(self):
        if self.output_path:
            path = Path(self.output_path)
            if (
                path.is_file()
                or Path(self.output_path).joinpath("results.json").is_file()
            ):
                print(f"File already exists at {path}. Results will be overwritten.")
                assert not path.is_file(), "File already exists"
            elif path.suffix in (".json", ".jsonl"):
                raise NotImplementedError("Not implemented")
            else:
                path.mkdir(parents=True, exist_ok=True)
        elif self.log_samples and not self.output_path:
            assert self.output_path, "Specify --output_path"

        if self.tokenizer_name_or_path == "":
            self.tokenizer_name_or_path = self.model_name_or_path

        if self.torch_dtype == "bfloat16":
            self.torch_dtype = torch.bfloat16
        elif self.torch_dtype == "float32":
            self.torch_dtype = torch.float32
        elif self.torch_dtype == "auto":
            self.torch_dtype = "auto"
        else:
            raise NotImplementedError("Torch dtype not implemented")

        if self.to_wandb and self.wandb_id == "":
            path = Path(self.model_name_or_path)
            assert path.joinpath(
                "run_args.yaml"
            ).is_file(), f"File not found at {path.joinpath('run_args.yaml')}"
            with open(path.joinpath("run_args.yaml"), "r", encoding="utf-8") as fread:
                all_args = yaml.load(fread, Loader=yaml.Loader)
            self.wandb_id = all_args["wandb_id"]
            self.wandb_project = all_args["wandb_project"]
            print(
                f"Read wandb info from {path.joinpath('run_args.yaml')}: {self.wandb_project}/{self.wandb_id}"
            )
        return

def _handle_non_serializable(o):
    if isinstance(o, np.int64) or isinstance(o, np.int32):
        return int(o)
    elif isinstance(o, set):
        return list(o)
    else:
        return str(o)

def save_results(
    args: LMEvalArguments, new_results: dict, new_tasks: list, prev_results=None
):
    path = Path(args.output_path)
    output_path_file = path.joinpath("results.json")

    if prev_results is not None:
        for k, old_results in prev_results.items():
            if k not in new_results:
                new_results[k] = old_results
            elif isinstance(old_results, dict):
                new_results[k] = {**old_results, **new_results[k]}
            elif isinstance(old_results, (str, float, int)):
                new_results[k] = old_results
            else:
                print("skipping", k, old_results)

    if args.log_samples:
        samples = new_results.pop("samples")

    dumped = json.dumps(
        new_results, indent=2, default=_handle_non_serializable, ensure_ascii=False
    )

    output_path_file.open("w", encoding="utf-8").write(dumped)
    if args.log_samples:
        for task_name, config in new_results["configs"].items():
            if task_name not in new_tasks:
                print(f"Task {task_name} not in new_tasks: {new_tasks}")
                continue
            filename = path.joinpath(f"{task_name}.jsonl")
            samples_dumped = json.dumps(
                samples[task_name],
                indent=2,
                default=_handle_non_serializable,
                ensure_ascii=False,
            )
            filename.write_text(samples_dumped, encoding="utf-8")
    return new_results

def get_performance(args: LMEvalArguments, all_results, all_tasks):
    metrics = {}
    all_averages = []
    openllm_averages = []
    classification_average = []
    generation_average = []
    for task, task_result in all_results["results"].items():
        if task in all_tasks:
            task_result_cleaned = {}
            for k, v in task_result.items():
                if k == "alias":
                    continue
                k = k.replace(",none", "")
                task_result_cleaned[k] = v

                if k != TASK_TO_METRIC[task]:
                    continue
                all_averages.append(v)
                if task in OPENLLM_TASKS:
                    openllm_averages.append(v)
                if task in GEN_TASKS:
                    generation_average.append(v)
                else:
                    classification_average.append(v)
            metrics[task] = task_result_cleaned
    metrics["openllm_average"] = np.mean(openllm_averages).item()
    metrics["classification_average"] = np.mean(classification_average).item()
    metrics["generation_average"] = np.mean(generation_average).item()
    metrics["all_average"] = np.mean(all_averages).item()

    path = Path(args.output_path)
    output_path_file = path.joinpath("performance.json")
    dumped = json.dumps(
        metrics, indent=2, default=_handle_non_serializable, ensure_ascii=False
    )
    output_path_file.open("w", encoding="utf-8").write(dumped)
    return metrics


In [9]:
def main(args: LMEvalArguments):
    print(f"Eval parameters {args}")

    task_manager = TaskManager(args.verbosity, include_path=None)

    loaded_model, loaded_tokenizer = FastLanguageModel.from_pretrained(
        model_name=args.model_name_or_path,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )

    if "stablelm" in loaded_tokenizer.name_or_path:
        print(
            "Setting pad token id to 100288 assuming you are using StableLM tokenizer"
        )
        loaded_tokenizer.pad_token_id = 100288

    loaded_model = loaded_model.eval()

    lm = lm_eval.api.registry.get_model(args.model).create_from_arg_string(
        "",
        {
            "pretrained": loaded_model,
            "tokenizer": loaded_tokenizer,
            "trust_remote_code": True,
            "batch_size": args.batch_size,
            "max_batch_size": None,
            "device": None,
        },
    )

    start_time = time.time()

    prev_results = None
    for task in ALL_TASKS:
        print(f"Running task {task} with {TASK_TO_NUM_SHOT[task]} shots")
        new_results = evaluator.simple_evaluate(
            model=lm,
            tasks=[task],
            batch_size=args.batch_size,
            num_fewshot=TASK_TO_NUM_SHOT[task],
            log_samples=True,
            gen_kwargs=None,
            task_manager=task_manager,
        )
        prev_results = save_results(
            args, new_results=new_results, new_tasks=[task], prev_results=prev_results
        )
    time_taken = time.time() - start_time

    all_tasks = ALL_TASKS
    performance = get_performance(args, prev_results, all_tasks)
    print(json.dumps(performance, indent=2, ensure_ascii=False))
    print(f"Time taken: {time_taken} seconds")



    if args.wandb_id != "":
        wandb.init(project=args.wandb_project, id=args.wandb_id, resume=True)
        wandb_perf = {f"lm_eval/{k}": v for k, v in performance.items()}
        wandb_perf["lm_eval/time_taken"] = time_taken
        wandb.log(wandb_perf)
        wandb.finish()
    return

if __name__ == "__main__":
    args = LMEvalArguments(
        model="hf",
        model_name_or_path="unsloth/llama-2-7b-chat-bnb-4bit",
        model_revision="main",
        tokenizer_name_or_path="",
        tokenizer_revision="main",
        attn_implementation=None,
        torch_dtype="auto",
        trust_remote_code=True,
        batch_size=1,
        output_path="",
        log_samples=True,
        verbosity="DEBUG",
        to_wandb=False,
        wandb_project="",
        wandb_id="",
    )


    main(args)

Eval parameters LMEvalArguments(model='hf', model_name_or_path='unsloth/llama-2-7b-chat-bnb-4bit', model_revision='main', tokenizer_name_or_path='', tokenizer_revision='main', attn_implementation=None, torch_dtype='auto', trust_remote_code=True, batch_size=1, output_path='', log_samples=True, verbosity='DEBUG', to_wandb=False, wandb_project='', wandb_id='')


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

...

skipping git_hash None
skipping upper_git_hash None
Task bbh_cot_fewshot_logical_deduction_three_objects not in new_tasks: ['bbh_cot_fewshot_reasoning_about_colored_objects']
Running task bbh_cot_fewshot_temporal_sequences with None shots


Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

INFO:lm-eval:Building contexts for bbh_cot_fewshot_temporal_sequences on rank 0...
100%|██████████| 250/250 [00:00<00:00, 703.91it/s]
INFO:lm-eval:Running generate_until requests
Running generate_until requests: 100%|██████████| 250/250 [34:42<00:00,  8.33s/it]


skipping git_hash None
skipping upper_git_hash None
Task bbh_cot_fewshot_logical_deduction_three_objects not in new_tasks: ['bbh_cot_fewshot_temporal_sequences']
Task bbh_cot_fewshot_reasoning_about_colored_objects not in new_tasks: ['bbh_cot_fewshot_temporal_sequences']
{
  "bbh_cot_fewshot_logical_deduction_three_objects": {
    "exact_match,get-answer": 0.636,
    "exact_match_stderr,get-answer": 0.030491555220405562
  },
  "bbh_cot_fewshot_reasoning_about_colored_objects": {
    "exact_match,get-answer": 0.372,
    "exact_match_stderr,get-answer": 0.030630325944558317
  },
  "bbh_cot_fewshot_temporal_sequences": {
    "exact_match,get-answer": 0.12,
    "exact_match_stderr,get-answer": 0.020593600596839963
  },
  "openllm_average": NaN,
  "classification_average": NaN,
  "generation_average": 0.37600000000000006,
  "all_average": 0.37600000000000006
}
Time taken: 5768.198976993561 seconds


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
